<a href="https://colab.research.google.com/github/NimaZah/Ethnic-Conflict/blob/main/EthnicConflictipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#/*===========================================================================
# This code is written in R language: 
# The code has incompatibility with colab envirenment. Please, first set up the
# editing envirenment
#---------------------------------------------------------------------------
#  Creation Date:      February 6, 2020
#  Do-file version:    06
#  Output:             Graphs
#  In collaboration with Anahita Zahedinameghi 
#===========================================================================*/

#/*=========================================================================
#                                   0: Program set up
#===========================================================================*/
# NOTES: 

# 1. Users can access data using the following address:

# "https://raw.githubusercontent.com/NimaZah/Ethnic-Conflict/main/data.csv"
# 
#  

#clear environment
rm(list=ls())

# Load libraries
library(QCA)
library(foreign)
library(KRLS)
require(pROC)

# Load data
x <- read.csv("https://raw.githubusercontent.com/NimaZah/Ethnic-Conflict/main/data.csv", 
              header = TRUE, sep=";")
head(x)



In [ ]:
# QCA analysis of necessary conditions
condsfull < -c("polx", "oust", "ruler", "petrol", "precon", "tiny", "instab", "tekcon",
  "conc", "xpoor", "havtek")
superSubset(x, outcome = "onset", neg.out = FALSE, conditions = condsfull,
  relation = "necessity", incl.cut = 0.95, cov.cut = 0.5,
  use.tilde = FALSE, use.letters = FALSE)
superSubset(x, outcome = "onset", neg.out = FALSE, conditions = condsfull,
  relation = "necessity", incl.cut = 0.95, cov.cut = 0.2,
  use.tilde = FALSE, use.letters = FALSE)

# QCA sufficiency analysis with all 11 conditions(not reported in the report)
condsfull < -c("polx", "oust", "ruler", "petrol", "precon", "tiny", "instab",
  "tekcon", "conc", "xpoor", "havtek")
tt < -truthTable(x, outcome = "onset", conditions = condsfull, incl.cut1 = 0.7,
  sort.by = c("incl", "n"), complete = TRUE)
tt

ILR < -rownames(tt$tt[(tt$tt$POLX == 1 | tt$tt$OUST == 1) & tt$tt$RULER == 1, ])
tt$tt[ILR, ]
sol.com < -eqmcc(tt, details = TRUE)
sol.com
sol.par < -eqmcc(tt, include = "?", omit = ILR, details = TRUE)
sol.par
sol.int < -eqmcc(tt, include = "?", omit = ILR, direxp = c(1, 1, 0,
    1, 1, 0,
    1, 1, 1,
    1, 1),
  details = TRUE)
sol.int
factorize(sol.int)

In [ ]:
# Replication of Table II
conds < -c("polx", "oust", "ruler", "petrol", "precon", "tiny", "instab", "tekcon")
tt < -truthTable(x, outcome = "onset", conditions = conds, incl.cut1 = 0.7,
  sort.by = c("incl", "n"), complete = TRUE)
tt
ILR < -rownames(tt$tt[(tt$tt$POLX == 1 | tt$tt$OUST == 1) & tt$tt$RULER == 1, ])
tt$tt[ILR, ]
sol.com < -eqmcc(tt, details = TRUE)
sol.com
sol.par < -eqmcc(tt, include = "?", omit = ILR, details = TRUE)
sol.par
sol.int < -eqmcc(tt, include = "?", omit = ILR, direxp = c(1, 1, 0, 1, 1, 0, 1, 1),
  details = TRUE)
sol.int
factorize(sol.int)

# Replication of Table IV, in -sample predictions

# all positive predictions QCA
which(x$ruler == 0 & x$tiny == 0 & ((x$polx == 1 & x$precon == 1) | (x$oust == 1 &
    x$instab == 1 &
    x$polx == 0) |
  (x$petrol == 0 & x$instab == 1 & x$tekcon == 1) |
  (x$polx == 1 & x$petrol == 1 & x$instab == 1 & x$tekcon == 0)))
# all true predictions QCA
which(x$onset == 1 & x$ruler == 0 & x$tiny == 0 & ((x$polx == 1 & x$precon == 1) |
  (x$oust == 1 & x$instab == 1 & x$polx == 0) |
  (x$petrol == 0 & x$instab == 1 & x$tekcon == 1) |
  (x$polx == 1 & x$petrol == 1 & x$instab == 1 & x$tekcon == 0)))

# in -sample prediction logit(replication of table VII online appendix)
glm.fit < -glm(onset~polx + oust + ruler + tiny + precon + petrol + instab + tekcon, data = x, family = binomial)
summary(glm.fit)
glm.pred < -predict(glm.fit, type = "response")
glm.pred
roc.fit < -roc(response = x$onset, predictor = glm.pred)
plot(roc.fit)
coords(roc.fit, "b", ret = "sensitivity", best.method = "closest.topleft")

In [ ]:
# Replication of Table IV, out - of -sample predictions

# out - of -sample prediction logit(also replication of table VII online appendix)
training.df < -read.csv("Prediction - estimation data 1990-2004.csv", header = TRUE, sep = ";")
test.df < -read.csv("Prediction - prediction data 2005-2009.csv", header = TRUE, sep = ";")
training.fit < -glm(onset~polx + oust + ruler + tiny + precon + petrol +
  instab + tekcon, data = training.df, family = binomial)
summary(training.fit)
test.pred < -predict(training.fit, type = "response", newdata = test.df)
test.pred
roc.fit < -roc(response = test.df$onset, predictor = test.pred)
plot(roc.fit)
coords(roc.fit, "b", ret = "sensitivity", best.method = "closest.topleft")

# out - of -sample prediction QCA

x2 < -read.csv("Prediction - estimation data 1990-2004.csv", header = TRUE, sep = ";")
conds < -c("polx", "oust", "ruler", "petrol", "precon", "tiny", "instab", "tekcon")
tt < -truthTable(x2, outcome = "onset", conditions = conds, incl.cut1 = 0.7,
  sort.by = c("incl", "n"), complete = TRUE)
ILR < -rownames(tt$tt[(tt$tt$POLX == 1 | tt$tt$OUST == 1) & tt$tt$RULER == 1, ])
sol.int2 < -eqmcc(tt, include = "?", omit = ILR, direxp = c(1, 1, 0, 1, 1, 0, 1, 1), details = TRUE)
sol.int2

In [ ]:
# load test data
x3 < -read.csv("Prediction - prediction data 2005-2009.csv", header = TRUE, sep = ";")
# all positive predictions based on sol.int2
which(x3$ruler == 0 & x3$tiny == 0 & ((x3$polx == 1 & x3$precon == 1) |
  (x3$oust == 1 & x3$instab == 1 & x3$polx == 0) |
  (x3$petrol == 0 & x3$instab == 1 & x3$tekcon == 1) |
  (x3$polx == 1 & x3$petrol == 1 & x3$instab == 1 & x3$tekcon == 0)))
# all true predictions based on sol.int2
which(x3$onset == 1 & x3$ruler == 0 & x3$tiny == 0 & ((x3$polx == 1 & x3$precon == 1) |
  (x3$oust == 1 & x3$instab == 1 & x3$polx == 0) |
  (x3$petrol == 0 & x3$instab == 1 & x3$tekcon == 1) |
  (x3$polx == 1 & x3$petrol == 1 & x3$instab == 1 & x3$tekcon == 0)))

In [ ]:
# out - of -sample prediction QCA at threshold 0.75(QCA2)
x4 < -read.csv("Prediction - estimation data 1990-2004.csv", header = TRUE, sep = ";")
conds < -c("polx", "oust", "ruler", "petrol", "precon", "tiny", "instab", "tekcon")
tt < -truthTable(x4, outcome = "onset", conditions = conds, incl.cut1 = 0.75,
  sort.by = c("incl", "n"), complete = TRUE)
ILR < -rownames(tt$tt[(tt$tt$POLX == 1 | tt$tt$OUST == 1) & tt$tt$RULER == 1, ])
sol.int3 < -eqmcc(tt, include = "?", omit = ILR, direxp = c(1, 1, 0, 1, 1, 0, 1, 1), details = TRUE)
sol.int3

In [ ]:
# load test data
x4 < -read.csv("Prediction - prediction data 2005-2009.csv", header = TRUE, sep = ";")
# all positive predictions based on sol.int3
which(x4$instab == 1 & x4$ruler == 0 & x4$tiny == 0 & ((x4$oust == 1 & x4$tekcon == 1) |
    (x4$polx == 0 & x4$oust == 1) |
    (x4$polx == 0 & x4$tekcon == 1) |
    (x4$polx == 1 & x4$petrol == 1 & x4$tekcon == 0)) |
  (x4$polx == 1 & x4$ruler == 0 & x4$precon == 1 & x4$tiny == 0))
# all true predictions based on sol.int3
which(x4$onset == 1 & (x4$instab == 1 & x4$ruler == 0 & x4$tiny == 0 & ((x4$oust == 1 & x4$tekcon == 1) |
    (x4$polx == 0 & x4$oust == 1) |
    (x4$polx == 0 & x4$tekcon == 1) |
    (x4$polx == 1 & x4$petrol == 1 & x4$tekcon == 0)) |
  (x4$polx == 1 & x4$ruler == 0 & x4$precon == 1 & x4$tiny == 0)))

In [ ]:
# Replication of Table VI in the online appendix(robustness test polx2)
x < -read.csv("Replication data incentives and opportunities.csv", header = TRUE, sep = ";")
head(x)
conds < -c("polx2", "oust", "ruler", "petrol", "precon", "tiny", "instab", "tekcon")
tt < -truthTable(x, outcome = "onset", conditions = conds, incl.cut1 = 0.7,
  sort.by = c("incl", "n"), complete = TRUE)
tt
ILR < -rownames(tt$tt[(tt$tt$POLX == 1 | tt$tt$OUST == 1) & tt$tt$RULER == 1, ])
tt$tt[ILR, ]
sol.int < -eqmcc(tt, include = "?", omit = ILR, direxp = c(1, 1, 0, 1, 1, 0, 1, 1), details = TRUE)
sol.int

In [ ]:
# Replication of Table IX in the online appendix('peace'
  analysis)
condsfull < -c("polx", "oust", "ruler", "petrol", "precon", "tiny", "instab", "tekcon",
  "conc", "xpoor", "havtek", "xrich70", "xrich80")
superSubset(x, outcome = "onset", neg.out = TRUE, conditions = condsfull,
  relation = "necessity", incl.cut = 0.95, cov.cut = 0.5, use.tilde = FALSE,
  use.letters = FALSE)

# Replication of Table X('peace'
  analysis) in the online appendix, Model 1
conds1 < -c("oust", "precon", "tekcon", "tiny", "ruler")
tt < -truthTable(x, outcome = "onset", neg.out = TRUE, conditions = conds1,
  incl.cut1 = 0.9, n.cut = 5,
  sort.by = c("incl", "n"), complete = TRUE)
tt
sol.com < -eqmcc(tt, details = TRUE)
sol.com
sol.par < -eqmcc(tt, include = "?", details = TRUE)
sol.par
sol.int < -eqmcc(tt, include = "?", direxp = c(0, 0, 0, 1, 1), details = TRUE)
sol.int

# Replication of Table X('peace'
  analysis) in the online appendix, Model 2
conds3 < -c("oust", "precon", "tekcon", "petrol", "tiny", "ruler")
tt < -truthTable(x, outcome = "onset", neg.out = TRUE, conditions = conds3,
  incl.cut1 = 0.9, n.cut = 5,
  sort.by = c("incl", "n"), complete = TRUE)
tt
sol.com < -eqmcc(tt, details = TRUE)
sol.com
sol.par < -eqmcc(tt, include = "?", details = TRUE)
sol.par
sol.int < -eqmcc(tt, include = "?", direxp = c(0, 0, 0, 0, 1, 1), details = TRUE)
sol.int

In [ ]:
# Replication of tables XI and XII in the online appendix(interaction terms)
# Conflict trap
glm.fit < -glm(onset~polx + oust + ruler + tiny + precon + petrol + instab +
  tekcon + polx_precon, data = x, family = binomial)
summary(glm.fit)
glm.pred < -predict(glm.fit, type = "response")
glm.pred
roc.fit < -roc(response = x$onset, predictor = glm.pred)
plot(roc.fit)
coords(roc.fit, "b", ret = "sensitivity", best.method = "closest.topleft")
pforconflicttrap < -cbind(response = x$onset, predictor = glm.pred)
write.table(pforconflicttrap, file = "pforconflicttrap.csv", sep = ",", row.names = F)

In [ ]:
# Conflict trap with necessary condition not tiny & not ruler
glm.fit < -glm(onset~polx + oust + nottinynotruler + precon + petrol +
  instab + tekcon + ntnr_polx_precon + ntnr_polx +
  ntnr_precon + polx_precon, data = x, family = binomial)
summary(glm.fit)
glm.pred < -predict(glm.fit, type = "response")
glm.pred
roc.fit < -roc(response = x$onset, predictor = glm.pred)
plot(roc.fit)
coords(roc.fit, "b", ret = "sensitivity", best.method = "closest.topleft")
pfornecconconflicttrap < -cbind(response = x$onset, predictor = glm.pred)
write.table(pfornecconconflicttrap, file = "pfornecconconflicttrap.csv", sep = ",",
  row.names = F)

In [ ]:
# Bad neighborhood
glm.fit < -glm(onset~polx + oust + ruler + tiny + precon + petrol +
  instab + tekcon + tekcon_instab, data = x, family = binomial)
summary(glm.fit)
glm.pred < -predict(glm.fit, type = "response")
glm.pred
roc.fit < -roc(response = x$onset, predictor = glm.pred)
plot(roc.fit)
coords(roc.fit, "b", ret = "sensitivity", best.method = "closest.topleft")
pforneighborhood < -cbind(response = x$onset, predictor = glm.pred)
write.table(pforneighborhood, file = "pforneighborhood.csv", sep = ",", row.names = F)

# Bad neighborhood with necessary condition not tiny & not ruler
glm.fit < -glm(onset~polx + oust + nottinynotruler + precon + petrol +
  instab + tekcon + ntnr_tekcon_instab + ntnr_tekcon +
  ntnr_instab + tekcon_instab, data = x, family = binomial)
summary(glm.fit)
glm.pred < -predict(glm.fit, type = "response")
glm.pred
roc.fit < -roc(response = x$onset, predictor = glm.pred)
plot(roc.fit)
coords(roc.fit, "b", ret = "sensitivity", best.method = "closest.topleft")
pfornecconneighborhood < -cbind(response = x$onset, predictor = glm.pred)
write.table(pfornecconneighborhood, file = "pfornecconneighborhood.csv", sep = ",
    ",row.names=F)

In [ ]:
# Ousted rulers
glm.fit < -glm(onset~polx + oust + ruler + tiny + precon + petrol +
  instab + tekcon + oust_instab, data = x, family = binomial)
summary(glm.fit)
glm.pred < -predict(glm.fit, type = "response")
glm.pred
roc.fit < -roc(response = x$onset, predictor = glm.pred)
plot(roc.fit)
coords(roc.fit, "b", ret = "sensitivity", best.method = "closest.topleft")
pforoustedrulers < -cbind(response = x$onset, predictor = glm.pred)
write.table(pforoustedrulers, file = "pforoustedrulers.csv", sep = ",", row.names = F)

In [ ]:
# Resource curse
glm.fit < -glm(onset~polx + oust + ruler + tiny + precon + petrol + instab +
  tekcon + petrol_polx_instab + petrol_polx + petrol_instab +
  polx_instab, data = x, family = binomial)
summary(glm.fit)
glm.pred < -predict(glm.fit, type = "response")
glm.pred
roc.fit < -roc(response = x$onset, predictor = glm.pred)
plot(roc.fit)
coords(roc.fit, "b", ret = "sensitivity", best.method = "closest.topleft")
pforresourcecurse < -cbind(response = x$onset, predictor = glm.pred)
write.table(pforresourcecurse, file = "pforresourcecurse.csv", sep = ",", row.names = F)

In [ ]:
# All four paths
glm.fit < -glm(onset~polx + oust + ruler + tiny + precon + petrol +
  instab + tekcon + polx_precon + tekcon_instab + oust_instab +
  petrol_polx_instab + petrol_polx + petrol_instab +
  polx_instab, data = x, family = binomial)
summary(glm.fit)
glm.pred < -predict(glm.fit, type = "response")
glm.pred
roc.fit < -roc(response = x$onset, predictor = glm.pred)
plot(roc.fit)
coords(roc.fit, "b", ret = "sensitivity", best.method = "closest.topleft")
pforallpaths < -cbind(response = x$onset, predictor = glm.pred)
write.table(pforallpaths, file = "pforallpaths.csv", sep = ",", row.names = F)